In [47]:
import pygame
import random

pygame.init()

WINDOW_WIDTH = 800
WINDOW_HEIGHT = 800
window = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Intersection Simulation")

BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0, 100)
GREEN = (0, 255, 0, 100)
YELLOW = (255, 255, 0, 100)
GRAY = (60, 60, 60)

ambient_1 = pygame.image.load('assets/ambient_1.png')
ambient_2 = pygame.image.load('assets/ambient_2.png')
ambient_3 = pygame.image.load('assets/ambient_3.png')
ambient_4 = pygame.image.load('assets/ambient_4.png')

# Load image, resize it to be 370x370 and put it in the right corner
ambient_1 = pygame.transform.scale(ambient_1, (370, 370))
ambient_2 = pygame.transform.scale(ambient_2, (370, 370))
ambient_3 = pygame.transform.scale(ambient_3, (370, 370))
ambient_4 = pygame.transform.scale(ambient_4, (370, 370))
window.blit(ambient_1, (0, 0))
window.blit(ambient_2, (430, 0))
window.blit(ambient_3, (430, 430))
window.blit(ambient_4, (0, 430))

class Car:
    def __init__(self, x, y, direction):
        self.x = x
        self.y = y
        self.direction = direction
        self.speed = 2
        self.width = 20
        self.length = 40
        self.stopped = False
        self.color = (random.randint(1, 255), random.randint(1, 255), random.randint(1, 255))
        self.turn_right = random.choice([False, True])

    def move(self):
        if not self.stopped:
            if self.direction == 'up':
                self.y -= self.speed
            elif self.direction == 'down':
                self.y += self.speed
            elif self.direction == 'left':
                self.x -= self.speed
            elif self.direction == 'right':
                self.x += self.speed

    def draw(self, win):
        if self.direction == 'up':
            pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.length))
        elif self.direction == 'down':
            pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.length))
        elif self.direction == 'left':
            pygame.draw.rect(win, self.color, (self.x, self.y, self.length, self.width))
        else:
            pygame.draw.rect(win, self.color, (self.x, self.y, self.length, self.width))

    def stop(self):
        self.stopped = True

    def can_move(self, other_cars):
        for other_car in other_cars:
            if other_car.stopped:
                if self.direction == other_car.direction == 'up':
                    if other_car.y + other_car.length + 4 <= self.y <= other_car.y + other_car.length + 6:
                        return False
                elif self.direction == other_car.direction == 'down':
                    if other_car.y - self.length - 6 <= self.y <= other_car.y - self.length - 4:
                        return False
                elif self.direction == other_car.direction == 'left':
                    if other_car.x + other_car.length + 4 <= self.x <= other_car.x + other_car.length + 6:
                        return False
                elif self.direction == other_car.direction == 'right':
                    if other_car.x - self.length - 6 <= self.x <= other_car.x - self.length - 4:
                        return False
        return True

    def turn_or_straight(self):
        if self.turn_right == True:
            if self.direction == 'up' and self.y <= WINDOW_HEIGHT//2:
                self.direction = 'right'
                self.x += self.length // 2
                self.y = WINDOW_HEIGHT//2 + 5
            elif self.direction == 'down' and self.y + self.length >= WINDOW_HEIGHT//2:
                self.direction = 'left'
                self.x -= self.length // 2
                self.y = WINDOW_HEIGHT//2 - 20 - 4
            elif self.direction == 'left' and self.x <= WINDOW_WIDTH//2:
                self.direction = 'up'
                self.x = WINDOW_WIDTH//2 + 5
                self.y -= self.length // 2
            elif self.direction == 'right' and self.x + self.length >= WINDOW_WIDTH//2:
                self.direction = 'down'
                self.x = WINDOW_WIDTH//2 - 20 - 4
                self.y += self.length // 2

def main():
    run = True
    clock = pygame.time.Clock()

    cars = []
    prev_time = 0
    stoplight_NS = GREEN
    stoplight_EW = RED

    while run:
        clock.tick(30)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
        time = pygame.time.get_ticks() // 1000
        if time != prev_time:
            if time % 2 == 0:
                direction = random.choice(['up', 'down', 'left', 'right'])
                if direction == 'up':
                    cars.append(Car(WINDOW_WIDTH//2 + 5, WINDOW_HEIGHT, direction))
                elif direction == 'down':
                    cars.append(Car(WINDOW_WIDTH//2 - 20 - 4, 0, direction))
                elif direction == 'left':
                    cars.append(Car(WINDOW_WIDTH, WINDOW_HEIGHT//2 - 20 - 4, direction))
                elif direction == 'right':
                    cars.append(Car(0, WINDOW_HEIGHT//2 + 5, direction))
            if time % 10 == 0 and stoplight_NS == GREEN:
                stoplight_NS = YELLOW
            elif time % 12 == 0 and stoplight_NS == YELLOW:
                stoplight_NS = RED
                stoplight_EW = GREEN
            elif time % 10 == 0 and stoplight_EW == GREEN:
                stoplight_EW = YELLOW
            elif time % 12 == 0 and stoplight_EW == YELLOW:
                stoplight_EW = RED
                stoplight_NS = GREEN
            prev_time = time
        
        for car in cars:
            if not car.stopped:
                if ((car.direction == 'up' and car.y == WINDOW_HEIGHT//2 + 50 and (stoplight_NS == RED or stoplight_NS == YELLOW)) or
                    (car.direction == 'down' and car.y + car.length == WINDOW_HEIGHT//2 - 50 and (stoplight_NS == RED or stoplight_NS == YELLOW)) or
                    (car.direction == 'left' and car.x == WINDOW_WIDTH//2 + 50 and (stoplight_EW == RED or stoplight_EW == YELLOW)) or
                    (car.direction == 'right' and car.x + car.length == WINDOW_WIDTH//2 - 50 and (stoplight_EW == RED or stoplight_EW == YELLOW))) or not car.can_move(cars):
                    car.stop()
                    print("here")
                else:
                    if ((car.direction == 'up' and car.y <= WINDOW_HEIGHT//2 + car.speed and car.y >= WINDOW_HEIGHT//2 - car.speed) or
                        (car.direction == 'down' and car.y + car.length >= WINDOW_HEIGHT//2 - car.speed and car.y + car.length <= WINDOW_HEIGHT//2 + car.speed) or
                        (car.direction == 'left' and car.x <= WINDOW_WIDTH//2 + car.speed and car.x >= WINDOW_WIDTH//2 - car.speed) or
                        (car.direction == 'right' and car.x + car.length >= WINDOW_WIDTH//2 - car.speed and car.x + car.length <= WINDOW_WIDTH//2 + car.speed)):
                        car.turn_or_straight()
                    car.move()
            elif car.stopped and ((car.direction in ['up', 'down'] and stoplight_NS == GREEN) or (car.direction in ['left', 'right'] and stoplight_EW == GREEN)):
                car.stopped = False
                car.move()

        cars = [car for car in cars if 0 <= car.x <= WINDOW_WIDTH and 0 <= car.y <= WINDOW_HEIGHT]

        # Draw intersection
        ## Draw road
        pygame.draw.line(window, GRAY, (0, WINDOW_HEIGHT//2), (WINDOW_WIDTH, WINDOW_HEIGHT//2), 60)
        pygame.draw.line(window, GRAY, (WINDOW_WIDTH//2, 0), (WINDOW_WIDTH//2, WINDOW_HEIGHT), 60)
        ## Draw lanes
        pygame.draw.line(window, WHITE, (0, WINDOW_HEIGHT//2 - 28), (WINDOW_WIDTH, WINDOW_HEIGHT//2 - 28), 1)
        pygame.draw.line(window, WHITE, (0, WINDOW_HEIGHT//2 + 28), (WINDOW_WIDTH, WINDOW_HEIGHT//2 + 28), 1)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 28, 0), (WINDOW_WIDTH//2 - 28, WINDOW_HEIGHT), 1)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 28, 0), (WINDOW_WIDTH//2 + 28, WINDOW_HEIGHT), 1)
        pygame.draw.line(window, WHITE, (0, WINDOW_HEIGHT//2), (WINDOW_WIDTH, WINDOW_HEIGHT//2), 4)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2, 0), (WINDOW_WIDTH//2, WINDOW_HEIGHT), 4)
        ## Draw crosswalks near the intersection
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 23, WINDOW_HEIGHT//2 - 50), (WINDOW_WIDTH//2 - 23, WINDOW_HEIGHT//2 - 30), 2)
        ## Cover intersection
        pygame.draw.rect(window, GRAY, (WINDOW_WIDTH//2 - 29, WINDOW_HEIGHT//2 - 29, 60, 60))
        ## Draw stoplights as stop lines
        pygame.draw.line(window, stoplight_NS, (WINDOW_WIDTH//2 - 27, WINDOW_HEIGHT//2 - 29 - 30), (WINDOW_WIDTH//2 - 2, WINDOW_HEIGHT//2 - 29 - 30), 5)
        pygame.draw.rect(window, stoplight_NS, (WINDOW_WIDTH//2 + 3, WINDOW_HEIGHT//2 + 31, 28, 5))
        pygame.draw.rect(window, stoplight_EW, (WINDOW_WIDTH//2 - 30 - 4, WINDOW_HEIGHT//2 + 3, 5, 28))
        pygame.draw.rect(window, stoplight_EW, (WINDOW_WIDTH//2 + 31, WINDOW_HEIGHT//2 - 29, 5, 28))
        
        # Draw cars
        for car in cars:
            car.draw(window)
        
        pygame.display.update()

    pygame.quit()

if __name__ == "__main__":
    main()

here
here
here
here
here
here


In [5]:
(255, 0, 0) - (100, 0, 0)

TypeError: unsupported operand type(s) for -: 'tuple' and 'tuple'